In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
# Loop through each class directory
for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    
    # Loop through each image in the class directory
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        
        # Read and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Unable to load image at path: {image_path}")
            continue  # Skip to the next image
        if image.size == 0:
            print(f"Error: Empty image at path: {image_path}")
            continue  # Skip to the next image
        image = cv2.resize(image, (224, 224))
        image = np.expand_dims(image, axis=-1)  # Add channel dimension
        images.append(image)
        
        # Assign the label based on the class directory
        label = int(class_name)
        labels.append(label)

In [ ]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print the shapes of the images and labels arrays
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = np.eye(2)[y_train]
y_test = np.eye(2)[y_test]

In [ ]:
# Data augmentation using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Reshape images to 3D tensors (height, width, channels)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
# Create ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build your model on top of the base ResNet50 model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
# Reshape input data to remove extra dimensions
X_train = X_train.squeeze()
y_train = y_train.squeeze()

# Train model
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=32), epochs=5, validation_data=(X_test, y_test))

model.save('D:/DATASET/Models/model_ballooning')

# Evaluate model
test_loss, test_acc = model.evaluate(test_datagen.flow(X_test, y_test, batch_size=32))
print("Test Accuracy:", test_acc)

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

In [ ]:
# Classification report and confusion matrix
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

def predict_image(image_path, model):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0) / 255.0

    # Predict the score
    prediction = model.predict(image)
    score = np.argmax(prediction)
    
    print("Predictions: ",prediction)
    
    # Define the NASH score mapping
    nash_scores = {
        0: "0: Ballooning absent",
        1: "1: Ballooning present"
    }
    
    # Get the NASH score corresponding to the maximum probability
    nash_score = nash_scores[score]
    
    # Print the NASH score
    print('Predicted NASH score:', nash_score)
    
    return score

# Load the model
model = load_model('D:/DATASET/Models/model_ballooning.h5')

In [ ]:
# Plot evaluation graphs for the input image
def plot_evaluation(image_path, model):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0) / 255.0

    # Predict the score
    prediction = model.predict(image)
    predicted_score = np.argmax(prediction)

    # Plot sample predictions
    plt.figure(figsize=(5, 5))
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Predicted Score: {predicted_score}")
    plt.axis('off')
    plt.show()

    # Confusion matrix
    y_true = [0]  # As we are plotting for only one image
    predicted_classes = [predicted_score]
    conf_matrix = confusion_matrix(y_true, predicted_classes)
    plt.figure(figsize=(4, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion Matrix")
    plt.show()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_true, predicted_classes)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Take input image path from the user
image_path = "D:/DATASET/CNN/ballooning/train/1/8_179_31.png"

# Plot evaluation graphs for the input image
plot_evaluation(image_path, model)